<div style="background:linear-gradient(135deg,#0ea5e9,#1d4ed8);padding:22px;border-radius:16px;color:white;box-shadow:0 10px 25px rgba(0,0,0,.18);">
  <div style="font-size:12px;letter-spacing:.12em;opacity:.85;text-transform:uppercase">Data Engineering</div>
  <div style="font-size:22px;font-weight:800;line-height:1.2;">Phase A — Data Modeling</div>
  <div style="margin-top:8px;font-size:14px;opacity:.95">Build a Gold star schema from Silver parquet: <code>dim_products</code> and <code>fact_order_items</code>.</div>
</div>

In [1]:
import os
import pandas as pd

# notebook is in: project_root/notebooks
PROJECT_ROOT = os.path.abspath("..")

INTERIM_DIR = os.path.join(PROJECT_ROOT, "data", "interim")
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")

os.makedirs(PROCESSED_DIR, exist_ok=True)

orders_df = pd.read_parquet(os.path.join(INTERIM_DIR, "orders.parquet"))
order_products_df = pd.read_parquet(os.path.join(INTERIM_DIR, "order_products.parquet"))
products_df = pd.read_parquet(os.path.join(INTERIM_DIR, "products.parquet"))
aisles_df = pd.read_parquet(os.path.join(INTERIM_DIR, "aisles.parquet"))
departments_df = pd.read_parquet(os.path.join(INTERIM_DIR, "departments.parquet"))

print(" Loaded interim data")
print("orders_df:", orders_df.shape)
print("order_products_df:", order_products_df.shape)
print("products_df:", products_df.shape)
print("aisles_df:", aisles_df.shape)
print("departments_df:", departments_df.shape)


 Loaded interim data
orders_df: (3421083, 7)
order_products_df: (33819106, 4)
products_df: (49688, 4)
aisles_df: (134, 2)
departments_df: (21, 2)


<div style="margin-top:14px;background:rgba(255,255,255,.75);backdrop-filter:blur(10px);-webkit-backdrop-filter:blur(10px);
border:1px solid rgba(59,130,246,.25);border-left:6px solid #2563eb;border-radius:14px;padding:16px;">
  <div style="font-weight:800;color:#0f172a;font-size:15px;">Step 1 — Build <code>dim_products</code></div>
  <ul style="margin-top:10px;color:#334155;font-size:13.5px;line-height:1.7;"><li>Join <code>products</code> with <code>aisles</code> and <code>departments</code>.</li>
<li>Enforce 1 row per <code>product_id</code> (dimension primary key).</li></ul>
  <div style="margin-top:10px;font-size:13px;opacity:.85">Output: <code>dim_products_df</code></div>
</div>

In [2]:
# Build dim_products with join validation + strict column set (dimension must be 1 row per product_id). #can_be in functions

# Fail fast if lookup tables are not unique (prevents silent row multiplication)
assert aisles_df["aisle_id"].is_unique, "aisles_df has duplicate aisle_id values."
assert departments_df["department_id"].is_unique, "departments_df has duplicate department_id values."
assert products_df["product_id"].is_unique, "products_df has duplicate product_id values."

dim_products_df = (
    products_df
    .merge(aisles_df, on="aisle_id", how="left", validate="many_to_one")
    .merge(departments_df, on="department_id", how="left", validate="many_to_one")
    .loc[:, ["product_id", "product_name", "aisle_id", "aisle", "department_id", "department"]]
)

print("dim_products_df:", dim_products_df.shape)
dim_products_df.head()
#add sanity check here 

dim_products_df: (49688, 6)


,product_id,product_name,aisle_id,aisle,department_id,department
0,1,Chocolate Sandwich Cookies,61,cookies cakes,19,snacks
1,2,All-Seasons Salt,104,spices seasonings,13,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,tea,7,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,frozen meals,1,frozen
4,5,Green Chile Anytime Sauce,5,marinades meat preparation,13,pantry


<div style="margin-top:14px;background:rgba(255,255,255,.75);backdrop-filter:blur(10px);-webkit-backdrop-filter:blur(10px);
border:1px solid rgba(59,130,246,.25);border-left:6px solid #2563eb;border-radius:14px;padding:16px;">
  <div style="font-weight:800;color:#0f172a;font-size:15px;">Step 2 — Dimension Quality Gates</div>
  <ul style="margin-top:10px;color:#334155;font-size:13.5px;line-height:1.7;"><li>Primary key uniqueness for <code>product_id</code>.</li>
<li>Completeness checks for joined labels (aisle / department).</li></ul>
  
</div>

In [3]:
# Cell 3 (REPLACE)
# Validate dimension primary key + join completeness.

# PK checks
assert dim_products_df["product_id"].isna().sum() == 0, "dim_products_df has null product_id."
assert dim_products_df["product_id"].is_unique, "dim_products_df product_id is not unique."

# Completeness checks
missing_aisle = dim_products_df["aisle"].isna().mean()
missing_dept = dim_products_df["department"].isna().mean()

print(f"Missing aisle rate: {missing_aisle:.6f}")
print(f"Missing department rate: {missing_dept:.6f}")

TOL = 0.001
assert missing_aisle <= TOL, "Too many missing aisles; check aisle_id join keys."
assert missing_dept <= TOL, "Too many missing departments; check department_id join keys."

print("dim_products_df validation passed.")
#function

Missing aisle rate: 0.000000
Missing department rate: 0.000000
dim_products_df validation passed.


<div style="margin-top:14px;background:rgba(255,255,255,.75);backdrop-filter:blur(10px);-webkit-backdrop-filter:blur(10px);
border:1px solid rgba(59,130,246,.25);border-left:6px solid #2563eb;border-radius:14px;padding:16px;">
  <div style="font-weight:800;color:#0f172a;font-size:15px;">Step 3 — Build <code>fact_order_items</code></div>
  <ul style="margin-top:10px;color:#334155;font-size:13.5px;line-height:1.7;"><li>Keep only fact keys + measures (no dimension attributes).</li>
<li>Preserve grain: 1 row per (<code>order_id</code>, <code>product_id</code>).</li></ul>
  <div style="margin-top:10px;font-size:13px;opacity:.85">Output: <code>fact_order_items_df</code></div>
</div>

In [4]:
# Build fact_order_items as a true fact table (keys + measures only). Do not include dimension attributes.

# Keep only keys + measures if present (no assumptions about columns)
keep_cols = [c for c in ["order_id", "product_id", "add_to_cart_order", "reordered"] if c in order_products_df.columns]
fact_order_items_df = order_products_df.loc[:, keep_cols].copy()

# Referential integrity: product_id in fact must exist in dim_products
missing_fk_rate = (~fact_order_items_df["product_id"].isin(dim_products_df["product_id"])).mean()
print(f"Missing dim_products FK rate in fact: {missing_fk_rate:.6f}")

TOL = 0.001
assert missing_fk_rate <= TOL, "Too many product_id values in fact not found in dim_products."

print("fact_order_items_df:", fact_order_items_df.shape)
fact_order_items_df.head()


Missing dim_products FK rate in fact: 0.000000
fact_order_items_df: (33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


<div style="margin-top:14px;background:rgba(255,255,255,.75);backdrop-filter:blur(10px);-webkit-backdrop-filter:blur(10px);
border:1px solid rgba(59,130,246,.25);border-left:6px solid #2563eb;border-radius:14px;padding:16px;">
  <div style="font-weight:800;color:#0f172a;font-size:15px;">Step 4 — Fact Quality Gates</div>
  <ul style="margin-top:10px;color:#334155;font-size:13.5px;line-height:1.7;"><li>Foreign key integrity: <code>product_id</code> must exist in <code>dim_products</code>.</li>
<li>Duplicate diagnostics on (<code>order_id</code>, <code>product_id</code>).</li></ul>
  
</div>

In [5]:
# Validate fact foreign keys and give duplication diagnostics.

for k in ["order_id", "product_id"]:
    assert fact_order_items_df[k].isna().sum() == 0, f"fact_order_items_df has null {k}."

dup_rate = fact_order_items_df.duplicated(subset=["order_id", "product_id"]).mean()
print(f"Duplicate rate on (order_id, product_id): {dup_rate:.6f}")

print("Unique orders:", fact_order_items_df["order_id"].nunique())
print("Unique products:", fact_order_items_df["product_id"].nunique())
print("Total rows (order-items):", len(fact_order_items_df))


Duplicate rate on (order_id, product_id): 0.000000
Unique orders: 3346083
Unique products: 49685
Total rows (order-items): 33819106


<div style="margin-top:14px;background:rgba(255,255,255,.75);backdrop-filter:blur(10px);-webkit-backdrop-filter:blur(10px);
border:1px solid rgba(59,130,246,.25);border-left:6px solid #2563eb;border-radius:14px;padding:16px;">
  <div style="font-weight:800;color:#0f172a;font-size:15px;">Step 5 — Fact Profiling</div>
  <ul style="margin-top:10px;color:#334155;font-size:13.5px;line-height:1.7;"><li>Report unique orders, products, and total line items.</li>
<li>Sanity-check scale before DS layers.</li></ul>
  
</div>

In [6]:
print("Unique orders:", fact_order_items_df["order_id"].nunique())
print("Unique products:", fact_order_items_df["product_id"].nunique())
print("Total rows (order-items):", len(fact_order_items_df))


Unique orders: 3346083
Unique products: 49685
Total rows (order-items): 33819106


<div style="margin-top:14px;background:rgba(255,255,255,.75);backdrop-filter:blur(10px);-webkit-backdrop-filter:blur(10px);
border:1px solid rgba(59,130,246,.25);border-left:6px solid #2563eb;border-radius:14px;padding:16px;">
  <div style="font-weight:800;color:#0f172a;font-size:15px;">Step 6 — Persist Gold Parquet</div>
  <ul style="margin-top:10px;color:#334155;font-size:13.5px;line-height:1.7;"><li>Write Gold tables to <code>data/processed</code>.</li>
<li>These outputs are the contract for downstream transaction mining.</li></ul>
  <div style="margin-top:10px;font-size:13px;opacity:.85">Output: <code>dim_products.parquet</code>, <code>fact_order_items.parquet</code></div>
</div>

In [7]:
# Write stable parquet outputs (explicit engine + compression) for downstream DS.

dim_products_df.to_parquet(
    os.path.join(PROCESSED_DIR, "dim_products.parquet"),
    index=False,
    engine="pyarrow",
    compression="snappy"
)

fact_order_items_df.to_parquet(
    os.path.join(PROCESSED_DIR, "fact_order_items.parquet"),
    index=False,
    engine="pyarrow",
    compression="snappy"
)

print("Saved processed tables to:", PROCESSED_DIR)


Saved processed tables to: /Users/imakdoun/PycharmProjects/instacart-retail-project/data/processed


In [8]:
# OUTPUT CHECK A
print("dim_products_df shape:", dim_products_df.shape)
print(dim_products_df.dtypes)
print(dim_products_df.head(3))

print("\nfact_order_items_df shape:", fact_order_items_df.shape)
print(fact_order_items_df.dtypes)
print(fact_order_items_df.head(3))


dim_products_df shape: (49688, 6)
product_id       int64
product_name       str
aisle_id         int64
aisle              str
department_id    int64
department         str
dtype: object
   product_id                          product_name  aisle_id  \
0           1            Chocolate Sandwich Cookies        61   
1           2                      All-Seasons Salt       104   
2           3  Robust Golden Unsweetened Oolong Tea        94   

               aisle  department_id department  
0      cookies cakes             19     snacks  
1  spices seasonings             13     pantry  
2                tea              7  beverages  

fact_order_items_df shape: (33819106, 4)
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtype: object
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327        

In [9]:
# OUTPUT CHECK B
# Dimension integrity validation

print("dim_products_df shape:", dim_products_df.shape)

# Primary key checks
print("Null product_id count:", dim_products_df["product_id"].isna().sum())
print("Is product_id unique?:", dim_products_df["product_id"].is_unique)

# Duplicate PK rows
dup_dim = dim_products_df.duplicated(subset=["product_id"]).sum()
print("Duplicate product_id rows:", dup_dim)

# Join completeness checks
missing_aisle = dim_products_df["aisle"].isna().mean()
missing_dept = dim_products_df["department"].isna().mean()

print("Missing aisle rate:", round(missing_aisle, 6))
print("Missing department rate:", round(missing_dept, 6))


dim_products_df shape: (49688, 6)
Null product_id count: 0
Is product_id unique?: True
Duplicate product_id rows: 0
Missing aisle rate: 0.0
Missing department rate: 0.0


In [10]:
# OUTPUT CHECK C
print("fact order_id nulls:", fact_order_items_df["order_id"].isna().sum())
print("fact product_id nulls:", fact_order_items_df["product_id"].isna().sum())

missing_fk = (~fact_order_items_df["product_id"].isin(dim_products_df["product_id"])).mean()
print("Missing product_id FK rate in fact:", round(missing_fk, 6))

dup_fact = fact_order_items_df.duplicated(subset=["order_id", "product_id"]).mean()
print("Duplicate rate on (order_id, product_id):", round(dup_fact, 6))

print("Unique orders:", fact_order_items_df["order_id"].nunique())
print("Unique products:", fact_order_items_df["product_id"].nunique())
print("Total fact rows:", len(fact_order_items_df))


fact order_id nulls: 0
fact product_id nulls: 0
Missing product_id FK rate in fact: 0.0
Duplicate rate on (order_id, product_id): 0.0
Unique orders: 3346083
Unique products: 49685
Total fact rows: 33819106


In [11]:
# OUTPUT CHECK D
import os
import pandas as pd

dim_pq = pd.read_parquet(os.path.join(PROCESSED_DIR, "dim_products.parquet"))
fact_pq = pd.read_parquet(os.path.join(PROCESSED_DIR, "fact_order_items.parquet"))

print("dim_products.parquet shape:", dim_pq.shape)
print("fact_order_items.parquet shape:", fact_pq.shape)

print("dim parquet columns:", dim_pq.columns.tolist())
print("fact parquet columns:", fact_pq.columns.tolist())


dim_products.parquet shape: (49688, 6)
fact_order_items.parquet shape: (33819106, 4)
dim parquet columns: ['product_id', 'product_name', 'aisle_id', 'aisle', 'department_id', 'department']
fact parquet columns: ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
